# DataSet e bibliotecas a serem usados no projeto

In [127]:
import pandas as pd
import jax.numpy as jnp
import numpy as np
import jax as jax
from typing import Callable
from sklearn.model_selection import train_test_split
import time
import timeit
import matplotlib.pyplot as plt

# user id | item id | rating | timestamp.
_df = pd.read_csv('ml-100k/u.data', delimiter='\t', header=None, names=['userId', 'movieId', 'rating', 'timeStamp'])
originalData = _df.pivot(index='userId', columns='movieId', values='rating')
print(_df.head(), "\n", originalData.shape)


   userId  movieId  rating  timeStamp
0     196      242       3  881250949
1     186      302       3  891717742
2      22      377       1  878887116
3     244       51       2  880606923
4     166      346       1  886397596 
 (943, 1682)


### Separação de dados de teste

De acordo com a documentação, os dados já estão ordenados de maneira aleatória, então podemos apenas pegar os primeiros 20% de entradas.

In [128]:
testData = _df[:int(0.2*len(_df))]
_df = _df[int(0.2*len(_df)):]

### Tratamento dos dados: transposição em matriz, preenchimento de dados faltantes, normalização

In [129]:
# Convertendo a lista de dados em uma tabela com usuários nas linhas, filmes nas colunas, contendo os ratings correspondentes.
trainData = _df.pivot(index='userId', columns='movieId', values='rating')
testData = testData.pivot(index='userId', columns='movieId', values='rating')

# Reindexando os dataframes separados para incluuir todos os userIds e movieIds do dataframe original
userIds = [_ for _ in range(1,944)]
movieIds = [_ for _ in range(1,1683)]
trainData = trainData.reindex(index=userIds, columns=movieIds)
testData = testData.reindex(index=userIds, columns=movieIds)


print(trainData.head(), '\n')
print("Novo formato do DataFrame de treino: ", trainData.shape, '\n')
print("Novo formato do DataFrame de teste: ", testData.shape, '\n')

# Preenchendo valores faltantes com o rating médio do filme correspondente
trainDataFilled = trainData.apply(lambda x: x.fillna(x.mean()), axis=0)
print("DataFrame with column mean filled:\n", trainDataFilled.head(), '\n')


movieId  1     2     3     4     5     6     7     8     9     10    ...  \
userId                                                               ...   
1         5.0   3.0   4.0   3.0   3.0   NaN   4.0   1.0   5.0   NaN  ...   
2         4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
userId                                                               
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5         NaN   NaN   NaN   NaN   NaN   NaN   N

In [130]:
def normalizeData(data:jnp.ndarray) -> tuple[jnp.ndarray, jnp.ndarray]:
    """
    Normaliza dados de um array subtraindo médias normalizando em relação ao desvio padrão 
    de cada feature.

    Args:
        data (jnp.ndarray): Dados não normalizdos
        
    Returns:
        tuple[jnp.ndarray, jnp.ndarray]: Dados normalizados de treino e teste
    """

    # Normalizando os dados: subtração da média de cada feature e mapeando para o intervalo [0,1]
    mean = jnp.nanmean(data, axis=0)
    std = jnp.nanstd(data, axis=0)

    data = data - mean
    data = data / std
    
    return data


# Normalizando os dados. 
trainDataFilled = normalizeData(jnp.array(trainDataFilled.values))
trainDataFilled


Array([[ 1.8474478e+00, -5.4083842e-01,  2.8511946e+00, ...,
                   nan,            nan,            nan],
       [ 1.7864378e-01, -1.4251887e-06,  0.0000000e+00, ...,
                   nan,            nan,            nan],
       [-1.1936216e-06, -1.4251887e-06,  0.0000000e+00, ...,
                   nan,            nan,            nan],
       ...,
       [ 1.8474478e+00, -1.4251887e-06,  0.0000000e+00, ...,
                   nan,            nan,            nan],
       [-1.1936216e-06, -1.4251887e-06,  0.0000000e+00, ...,
                   nan,            nan,            nan],
       [-1.1936216e-06,  5.4368358e+00,  0.0000000e+00, ...,
                   nan,            nan,            nan]], dtype=float32)

In [ ]:
# trainDataDf = pd.DataFrame(trainData)

# # Preenchendo valores faltantes com o rating médio do filme correspondente
# dfMovieMean = trainDataDf.apply(lambda x: x.fillna(x.mean()), axis=0)

# # Preenchendo valores faltantes com a média dos ratings dados pelo usuário
# dfUserMean = trainDataDf.apply(lambda x: x.fillna(x.mean()), axis=1)

# # Preenchendo valroes faltantes com zeros
# dfZeros = trainDataDf.fillna(0)

# trainDataMovieMean = jnp.array(dfMovieMean)
# trainDataUserMean = jnp.array(dfUserMean)
# trainDataZeros = jnp.array(dfZeros)

# print("DataFrame with column mean filled:\n", dfMovieMean.head(), '\n')
# print("DataFrame with row mean filled:\n", dfUserMean.head(), '\n')
# print("DataFrame with zeros filled:\n", dfZeros.head(), '\n')

# Definindo as funções para implementação do SVD, KNN e PCA

In [22]:
def reorder(eigenvalues:jnp.ndarray, eigenvectors:jnp.ndarray) -> tuple[jnp.ndarray, jnp.ndarray]:
    """ Reorders in descending order the eigenvalues and corresponding eigenvectors based on the eigenvectors values.

    Args:
        eigenvalues (jnp.ndarray): 1D array containing the eigenvalues.
        eigenvectors (jnp.ndarray): 2D array cotaining eigenvectors as columns.

    Returns:
        tuple[jnp.ndarray, jnp.ndarray]: Reordered eigenvalues and eigenvectors.
    """
    orderedIndices = jnp.argsort(eigenvalues, descending=True) 
    orderedEigenvalues = eigenvalues[orderedIndices] 
    orderedEigenvectors = eigenvectors[:, orderedIndices] 

    return orderedEigenvalues, orderedEigenvectors


def svd(data:jnp.ndarray) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """ From a data matrix, return its Singular Value Decomposition (SVD) 
    computed using the eigendecomposition of the data's covariance matrix.

    Args:
        data (jnp.ndarray): Rectangular matrix

    Returns:
        Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]: SVD of the data matrix, where data = U.S.V^T
    """

    # Computing the data matrix's right-singular vectors, as well as the singular values
    eigenvalues, rEigenvectors = jnp.linalg.eig(data.T @ data)
    eigenvalues, rEigenvectors = reorder(eigenvalues, rEigenvectors)
    Vt = jnp.real(rEigenvectors.T)    
    S = jnp.real(eigenvalues ** (1/2))

    # Computing the data matrix's left-singular vectors
    eigenvalues, lEigenvectors = jnp.linalg.eig(data @ data.T)
    eigenvalues, lEigenvectors = reorder(eigenvalues, lEigenvectors)
    U = jnp.real(lEigenvectors)

    
    return U, S, Vt

In [ ]:
def pca(data:jnp.ndarray, nbComponents:int, svd=True) -> jnp.ndarray:
    """
    A partir de um conjunto de dados, retorna a matriz de projeção para as k componentes principais.

    Args:
        data (jnp.ndaaray): Dados para reduzir dimensionalidade.
        nbComponents (int): Número de componentes desejadas na projeção.

    Returns:
        jnp.ndarray: Matriz de projeção.
    """

    covarianceMatrix = (1/len(data)) * data.T @ data
    
    if svd:
        # Realizando a decomposição em valores singulares. U terá vetores coluna ortogonais.  
        # A pricípio, o ordenamento não é necessário pois svd() já retorna os valores singulares ordenados.  
        U, S, Ut = jax.scipy.linalg.svd(covarianceMatrix) 
        projectionMatrix = U[:, :nbComponents]
    else:
        # Decomposição a partir de autovalores e autovetores.
        # Aqui, o ordenamento dos autovetores baseado nos autovalores é necessário.
        eigenvalues, eigenvectors = jnp.linalg.eig(covarianceMatrix)  
        indices = jnp.argsort(eigenvalues, descending=True)           
        projectionMatrix = eigenvectors[:, indices]  
        projectionMatrix = projectionMatrix[:, :nbComponents]
   

    return projectionMatrix

In [ ]:
def knn(xTrain:jnp.ndarray, yTrain:jnp.ndarray, xTest:jnp.ndarray, k:int, metric:Callable[[jnp.ndarray, jnp.ndarray], float]) -> jnp.ndarray:
    """ Implementa a classificação de um conjunto de dados a partir do algoritmo de K-Nearest Neighbors (KNN).

    Args:
        xTrain (jnp.ndarray): Dados de treino
        yTrain (jnp.ndarray): Labels para os dados de treino
        xTest (jnp.ndarray): Dados de teste
        k (int): Número de vizinhos mais próximos usados para a classificação.
        metric (Callable): Função que calcula distância entre dois pontos.

    Returns:
        jnp.ndarray: Array contendo as predições realizadas para o conjunto de dados de teste xTest.
    """
    
    # Implementação JAX-friendly do cálculo da matriz de distâncias para um conjunto de pontos de teste.
    # O cálculo da matriz é feito vetorizando duas vezes a função de métrica, para calcular dois a dois
    # as distâncias entre pontos de treino e de teste.
    distances = jax.vmap(lambda train_point: jax.vmap(metric, in_axes=(None, 0))(train_point, xTest))(xTrain)

    # Ordenando as distâncias entre pontos, e pegando os targets/labels dos k pontos mais próximos para cada ponto de treino
    sorted_indices = jnp.argsort(distances, axis=0)
    nearestNeighborsIndices = sorted_indices[:k, :]
    nearestNeighbors = yTrain[nearestNeighborsIndices].astype(int)

    # Contagem dos números de cada target presente na lista de vizinhos próximos e definição do target estimado para o ponto
    # de teste baseado numa voto de maioria.
    totalLabels = 3
    targetCounts = jax.vmap(lambda neighbors: jnp.bincount(neighbors, minlength=totalLabels, length=3))(nearestNeighbors.T) 
    most_common_classes = jnp.argmax(targetCounts, axis=1)
    
    return most_common_classes

In [ ]:
nbComponents = 3
# Realizando a decomposição em valores singulares. U terá vetores coluna ortogonais.  
# A pricípio, o ordenamento não é necessário pois svd() já retorna os valores singulares ordenados.  
U, S, Vt = jax.scipy.linalg.svd(dfZeros) 
projectionMatrix = U[:, :nbComponents]

# Testando a implementação da decomposição em valores principais a partir dos autovalores e autovetores da matriz de covariância dos dados.

Aqui geramos uma matriz aleatória, calculamos seus valores e vetores principais, e comparamos com a implementação nativa do JAX para fins de checagem de sanidade. 

Vemos que a matrix ortogonal de vetores principais não é a mesma obtida pela implementação do JAX, porém isso acontece somente devido a uma inversão do sinal de alguns dos vetores. Nas matrizes U e Ut, isso se traduz respectivamente em uma inversão de sinal de colunas ou linhas especificas, o que não altera as propriedades matemáticas da nossa decomposição. Portanto, a menos de desvios numéricos, temos uma implementação equivalente feita a partir dos autovetores e autovalores da matriz de dados. 

In [43]:
# Gerando os dados aleatórios para teste
key = jax.random.PRNGKey(0)
random_array = jax.random.uniform(key, (5, 4))
data = random_array
covarianceMatrix = (1/len(data)) * data.T @ data

# Calculando a SVD usando dois métodos diferentes: implementação nativa e usando autovalores e autovetores
U1, S1, Vt1 = jax.scipy.linalg.svd(covarianceMatrix) 
U2, S2, Vt2 = svd(covarianceMatrix) 

In [44]:
# Os dois arrays devem ser iguais, a menos de um sinal em cada coluna
print("Comparando U: ")
print(U1)
print(U2)

# Os dois arrays devem ser iguais, a menos de um sinal em cada linha
print("\n\nComparando Ut: ")
print(Vt1)
print(Vt2)

# Os valores principais devem ser exatamente iguais
print("\n\nComparando valores principais: ")
print(S1)
print(S2)

Comparando U: 
[[-0.5625489   0.7061049  -0.26226145 -0.34084198]
 [-0.45437548 -0.49096245  0.43556333 -0.60231495]
 [-0.4274326  -0.50450015 -0.71912175  0.21364647]
 [-0.5425707   0.07649231  0.47367406  0.6894916 ]]
[[ 0.56254894 -0.70610535  0.26226294  0.34084186]
 [ 0.4543755   0.4909616  -0.43556163  0.6023146 ]
 [ 0.42743263  0.5045007   0.71912086 -0.21364544]
 [ 0.54257077 -0.07649165 -0.4736765  -0.6894922 ]]


Comparando Ut: 
[[-0.56254894 -0.4543754  -0.4274326  -0.5425706 ]
 [ 0.706105   -0.49096242 -0.50450003  0.07649231]
 [-0.26226142  0.43556345 -0.719122    0.47367418]
 [-0.340842   -0.60231483  0.21364635  0.6894918 ]]
[[ 0.56254894  0.4543755   0.42743257  0.5425707 ]
 [-0.7061053   0.49096262  0.5044998  -0.07649182]
 [ 0.26226133 -0.435565    0.7191219  -0.47367254]
 [ 0.3408419   0.60231435 -0.21364391 -0.689493  ]]


Comparando valores principais: 
[1.2094074  0.1760837  0.10692322 0.05494076]
[1.2094078  0.17608385 0.10692321 0.05494073]
